In [ ]:
using DrWatson
@quickactivate "Simulation"
include(srcdir("TienGordon.jl"))

import .TienGordon as TG
using CairoMakie
using ProgressMeter

In [ ]:
TG.ν = 15E9
# TG.extend = true
τ = TG.set_τ(0.05)
TG.n = 200

x = TG.V0;
y = collect(0:0.001:4) .* 0.0001;

In [ ]:
z = Matrix{Float64}(undef, length(x), length(y))
progress = Progress(length(y), 1, "Working...")
for yi in 1:lastindex(y)
    next!(progress)
    for xi in 1:lastindex(x)
        z[xi, yi] = TG.IV₀(x[xi], y[yi])
    end
end

dz = Matrix{Float64}(undef, length(x) - 1, length(y))
for i in 1:lastindex(y)
    dz[:, i] = diff(z[:, i]) ./ diff(x)
end

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])

lines!(ax, x[1:end-1], dz[:, 5])
# plt = heatmap!(ax, x[1:end-1], y, dz)
# Colorbar(fig[1, 2], plt)

display(fig)

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1])

# lines!(ax, x[1:end-1], dz[:, 1])
plt = heatmap!(ax, x[1:end-1], y, dz)
Colorbar(fig[1, 2], plt)

display(fig)

In [ ]:
using JLD2
data_dir = joinpath(@__DIR__, "..", "data")
data = jldopen(data_dir * "/iv.jld2", "r")
ref_V0, ref_I0 = data["normalized"][τ]

fig = Figure()
ax = Axis(fig[1, 1])

lines!(ax, x[1:end], z[:, 10], color = :Blue, label = "Simulation")
lines!(ax, ref_V0, ref_I0, color = :Red, label = "Reference")

ax = Axis(fig[1, 2])
lines!(ax, x[1:end-1], dz[:, 10], color = :Blue, label = "Simulation")
lines!(ax, ref_V0[1:end-1], diff(ref_I0) ./ diff(ref_V0), color = :Red, label = "Reference")

display(fig)